In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("Gold") \
    .master('local[*]') \
    .config("spark.driver.maxResultSize", "6g") \
    .config("spark.driver.memory", "3g") \
    .config("spark.executor.memory", "3g") \
    .config('spark.dynamicAllocation.minExecutors' , '1') \
    .config('spark.dynamicAllocation.maxExecutors' , '2') \
    .config('spark.dynamicAllocation.enabled' , 'true') \
    .config("spark.hadoop.fs.s3a.access.key", 'sJ8IkEjav4gkDwjt2BxK') \
    .config("spark.hadoop.fs.s3a.secret.key", 'atZpF6WhWSD0tH2vSxylDHYpdOWZ11zdnSZ87ca8') \
    .config("spark.hadoop.fs.s3a.endpoint", 'http://127.0.0.1:9000') \
    .config("spark.hadoop.fs.s3a.impl", 'org.apache.hadoop.fs.s3a.S3AFileSystem') \
    .config("spark.hadoop.fs.s3a.path.style.access", 'true') \
    .config("spark.sql.extensions", 'io.delta.sql.DeltaSparkSessionExtension') \
    .config("spark.sql.catalog.spark_catalog", 'org.apache.spark.sql.delta.catalog.DeltaCatalog') \
    .getOrCreate()

In [3]:
attributes_path = "s3a://silver/attributes"
business_attributes_path = "s3a://silver/business_attributes"
business_category_path = "s3a://silver/business_category"
business_work_hours_path = "s3a://silver/business_work_hours"
business_path = "s3a://silver/business"
category_path = "s3a://silver/category"
review_path = "s3a://silver/reviews"

In [4]:
attributes_df = spark.read.format("delta").load(attributes_path)
business_attributes_df = spark.read.format("delta").load(business_attributes_path)
business_category_df = spark.read.format("delta").load(business_category_path)
business_work_hours_df = spark.read.format("delta").load(business_work_hours_path)
business_df = spark.read.format("delta").load(business_path)
category_df = spark.read.format("delta").load(category_path)
review_path = spark.read.format("delta").load(review_path)

In [5]:
attributes_df.createOrReplaceTempView("attributes")
business_attributes_df.createOrReplaceTempView("business_attributes")
business_category_df.createOrReplaceTempView("business_category")
business_work_hours_df.createOrReplaceTempView("business_work_hours")
business_df.createOrReplaceTempView("business")
category_df.createOrReplaceTempView("category")
review_path.createOrReplaceTempView("reviews")

#### Câu hỏi 1: Sự phân bổ số lượng nhà hàng theo state ?

In [6]:
sql_query_1 = spark.sql("""
SELECT 
    CASE
				WHEN state = 'AB' THEN CONCAT('CA-', state)
				ELSE CONCAT('US-', state)
		END AS state,
    COUNT(business_id) AS total_restaurants
FROM business
GROUP BY state
ORDER BY total_restaurants DESC;
""")

sql_query_1.show()

+-----+-----------------+
|state|total_restaurants|
+-----+-----------------+
|US-PA|            34039|
|US-FL|            26330|
|US-TN|            12056|
|US-IN|            11247|
|US-MO|            10913|
|US-LA|             9924|
|US-AZ|             9912|
|US-NJ|             8536|
|US-NV|             7715|
|CA-AB|             5573|
|US-CA|             5203|
|US-ID|             4467|
|US-DE|             2265|
|US-IL|             2145|
|US-TX|                4|
|US-CO|                3|
|US-WA|                2|
|US-MA|                2|
|US-HI|                2|
|US-MI|                1|
+-----+-----------------+
only showing top 20 rows



In [7]:
sql_query_1.write.format("delta").mode("overwrite").save("s3a://gold/restaurants_distribution_city_state")

Khi nhìn vào dữ liệu về sự phân bổ nhà hàng theo thành phố và bang, một bức tranh rõ nét về thị trường nhà hàng trên toàn khu vực bắt đầu xuất hiện. Các thành phố lớn thường có số lượng nhà hàng tập trung cao, cho thấy nhu cầu ẩm thực của dân cư nơi đây đa dạng và phong phú. Thành phố dẫn đầu về số lượng nhà hàng thường là những nơi có mật độ dân cư cao, khu du lịch phát triển hoặc những điểm nóng ẩm thực, thu hút cả khách du lịch và người dân địa phương.

Mặt khác, các thành phố nhỏ hơn có thể cho thấy tiềm năng thị trường đang phát triển hoặc sự độc quyền của một số nhà hàng nổi bật, giúp họ trở thành trung tâm ẩm thực trong khu vực. Đối với các chủ nhà hàng, việc xem xét sự phân bổ này có thể hỗ trợ trong việc quyết định mở rộng kinh doanh, nhắm tới những thị trường tiềm năng hoặc tận dụng cơ hội trong các thị trường đang nổi.

#### Câu hỏi 2: Các thành phố nào có nhiều nhà hàng được đánh giá 4 sao trở lên ?

In [10]:
sql_query_2 = spark.sql("""
SELECT b.name, b.city, b.stars AS avg_stars, b.review_count
FROM business b
WHERE b.stars >= 4.5 AND b.review_count > 100
ORDER BY b.review_count DESC;
""")

sql_query_2.show()

+--------------------+-------------+---------+------------+
|                name|         city|avg_stars|review_count|
+--------------------+-------------+---------+------------+
|Hattie B’s Hot Ch...|    Nashville|      4.5|        6093|
|Reading Terminal ...| Philadelphia|      4.5|        5721|
|Ruby Slipper - Ne...|  New Orleans|      4.5|        5193|
|  Commander's Palace|  New Orleans|      4.5|        4876|
|  Pappy's Smokehouse|  Saint Louis|      4.5|        3999|
|      Cochon Butcher|  New Orleans|      4.5|        3837|
|          Los Agaves|Santa Barbara|      4.5|        3834|
|Willie Mae's Scot...|  New Orleans|      4.5|        3582|
|               Zahav| Philadelphia|      4.5|        3065|
|  Bern's Steak House|        Tampa|      4.5|        2924|
|            Barbuzzo| Philadelphia|      4.5|        2893|
|      Peg Leg Porker|    Nashville|      4.5|        2878|
|         Café Amelie|  New Orleans|      4.5|        2756|
|        The Stillery|    Nashville|    

In [11]:
sql_query_2.write.format("delta").mode("overwrite").save("s3a://gold/city_more_than_4s")

Câu truy vấn này mở ra bức tranh về những nhà hàng không chỉ được đánh giá cao (từ 4.5 sao trở lên) mà còn nhận được sự quan tâm và đánh giá từ số lượng lớn khách hàng (hơn 100 bài đánh giá). Điều này chứng minh rằng các nhà hàng này không chỉ thu hút được sự yêu mến của thực khách mà còn để lại ấn tượng sâu sắc khiến họ sẵn sàng chia sẻ cảm nhận. Trong thế giới ẩm thực cạnh tranh gay gắt, việc duy trì mức đánh giá cao kèm theo lượng lớn phản hồi từ khách hàng không phải là điều dễ dàng.

Đối với mỗi thành phố, những nhà hàng trong danh sách này trở thành biểu tượng của chất lượng và sự xuất sắc. Họ là những nơi mà thực khách tìm đến không chỉ để thưởng thức bữa ăn ngon mà còn để trải nghiệm một dịch vụ vượt trên cả sự mong đợi. Những lời đánh giá, phản hồi tích cực cùng với lượng lớn lượt đánh giá cũng cho thấy những nhà hàng này đã tạo nên được dấu ấn trong lòng thực khách, biến họ thành khách hàng trung thành hoặc thậm chí là người hâm mộ.

Với số lượng lớn các nhận xét từ thực khách, các nhà hàng này còn trở thành trung tâm của những câu chuyện truyền tai, đánh giá tốt từ miệng đến miệng, góp phần vào sự tăng trưởng và danh tiếng của họ. Họ là ví dụ điển hình về việc kết hợp chất lượng món ăn, không gian ấn tượng và sự tận tâm trong phục vụ, góp phần tạo ra trải nghiệm đáng nhớ mà khách hàng sẵn sàng chia sẻ với cả thế giới. Đối với những nhà hàng khác, câu chuyện của họ không chỉ là sự ngưỡng mộ mà còn là nguồn cảm hứng để phấn đấu vươn xa.

#### Câu hỏi 3: Phân tích hiệu suất của các nhà hàng dựa trên thuộc tính đặc biệt

In [15]:
sql_query_3 = spark.sql("""
SELECT a.attribute_name AS attribute_name, COUNT(DISTINCT b.business_id) AS num_restaurants, AVG(r.stars) AS avg_rating
FROM attributes a
JOIN business_attributes ba ON a.attribute_id = ba.attribute_id
JOIN business b ON ba.business_id = b.business_id
JOIN reviews r ON b.business_id = r.business_id
GROUP BY a.attribute_name
HAVING COUNT(DISTINCT b.business_id) > 10
ORDER BY avg_rating DESC;
""")

sql_query_3.show()

+--------------------+---------------+------------------+
|      attribute_name|num_restaurants|        avg_rating|
+--------------------+---------------+------------------+
|   HairSpecializesIn|           1065|  4.29827597909455|
|                BYOB|           4451| 4.030709025457482|
|             Corkage|           3553| 4.008411484296035|
|WheelchairAccessible|          28953|3.9778505394391677|
|BusinessAcceptsBi...|          17430|3.9719716527702325|
|         DogsAllowed|          18284|3.9564637741220623|
|         Open24Hours|             39| 3.917988893296311|
|RestaurantsTableS...|          19982| 3.905326221212274|
|RestaurantsCounte...|             19|3.8985158443642196|
|           HappyHour|          15171|  3.89178393759635|
|             Smoking|           4567| 3.890970133129456|
|   ByAppointmentOnly|          42339| 3.886321580760788|
|         GoodForMeal|          29087|3.8339377183479657|
|      GoodForDancing|           4628|3.8301892417976053|
|             

In [16]:
sql_query_3.write.format("delta").mode("overwrite").save("s3a://gold/attribute_restaurants_rating")

Câu truy vấn này đưa chúng ta vào hành trình khám phá những đặc điểm hoặc thuộc tính nổi bật nhất mà các nhà hàng đang áp dụng để thu hút thực khách và duy trì mức đánh giá cao. Bằng cách phân tích các thuộc tính của nhà hàng (chẳng hạn như có không gian ngoài trời, phục vụ món ăn chay, hoặc có khu vực vui chơi cho trẻ em), chúng ta sẽ thấy những yếu tố nào thực sự tạo ra sự khác biệt trong trải nghiệm của khách hàng. Đây là một cuộc hành trình để xác định các thuộc tính không chỉ phổ biến (có ở hơn 10 nhà hàng) mà còn góp phần vào việc duy trì mức độ yêu thích và đánh giá cao từ thực khách.

Khi sắp xếp kết quả dựa trên điểm đánh giá trung bình, chúng ta thấy những thuộc tính nào thực sự nổi bật và tạo ấn tượng mạnh mẽ với khách hàng. Có thể một nhà hàng với không gian ngoài trời mát mẻ sẽ làm dịu mát thực khách trong những ngày hè nóng bức, hay một quán ăn với menu chay phong phú đã làm hài lòng những người theo chế độ ăn kiêng đặc biệt. Những thuộc tính này không chỉ thể hiện sự hiểu biết và quan tâm đến nhu cầu khách hàng mà còn trở thành yếu tố làm nên danh tiếng của nhà hàng.

Nhìn từ góc độ kinh doanh, các nhà hàng khác có thể xem xét và áp dụng những thuộc tính nổi bật này để cải thiện trải nghiệm khách hàng. Câu chuyện này không chỉ là câu chuyện về sự yêu thích mà còn là bài học về cách một thuộc tính nhỏ bé nhưng mang tính quyết định có thể làm tăng mức độ hài lòng và mang lại thành công bền vững cho một nhà hàng. Nó khẳng định rằng trong thế giới ẩm thực, chính những chi tiết làm nên sự khác biệt, và các thuộc tính mang lại giá trị thực sự là những "vũ khí bí mật" giúp nhà hàng trở nên đặc biệt và đáng nhớ.

#### Câu hỏi 4: Tìm kiếm các nhà hàng có đánh giá “hữu ích” cao nhất từ khách hàng

In [20]:
sql_query_4 = spark.sql("""
SELECT b.name, SUM(r.useful) AS total_useful_reviews
FROM business b
JOIN reviews r ON b.business_id = r.business_id
GROUP BY b.name
ORDER BY total_useful_reviews DESC;
""")

sql_query_4.show()

+--------------------+--------------------+
|                name|total_useful_reviews|
+--------------------+--------------------+
|           Starbucks|               19281|
|  Lulou's Restaurant|               14627|
|          McDonald's|               12522|
|   ditech Home Loans|               10219|
| Walmart Supercenter|                8710|
|  Whole Foods Market|                7679|
|             Dunkin'|                7100|
|Chipotle Mexican ...|                7002|
|Power Home Remode...|                6668|
|         Chick-fil-A|                5993|
|        Panera Bread|                5942|
|        Trader Joe's|                5870|
|        CVS Pharmacy|                5383|
|  Commander's Palace|                5320|
|           Taco Bell|                5292|
|      Planet Fitness|                5292|
|         First Watch|                5006|
|Mazzaro's Italian...|                4977|
|       The UPS Store|                4974|
|   Acme Oyster House|          

In [18]:
sql_query_4.write.format("delta").mode("overwrite").save("s3a://gold/restaurant_useful_reviews")


Câu truy vấn này đưa chúng ta vào một hành trình khám phá những nhà hàng không chỉ nhận được nhiều phản hồi từ thực khách mà còn được đánh giá là hữu ích nhất. Đối với các nhà hàng, các bài đánh giá hữu ích là một nguồn thông tin vô giá. Những nhận xét này có thể cung cấp cái nhìn sâu sắc, không chỉ về trải nghiệm thực khách mà còn phản ánh chính xác những điểm mạnh và yếu của nhà hàng từ góc nhìn người dùng.

Nhà hàng có tổng số bài đánh giá hữu ích cao thường là những địa điểm mà thực khách không ngần ngại chia sẻ kinh nghiệm của mình để giúp những người khác đưa ra quyết định đúng đắn. Điều này cho thấy mức độ ảnh hưởng của những nhà hàng này không chỉ trong cộng đồng địa phương mà còn trên các nền tảng đánh giá trực tuyến. Một đánh giá hữu ích có thể mô tả chi tiết về món ăn ngon nhất, khuyến nghị về cách thưởng thức, hoặc thậm chí chia sẻ về dịch vụ mà khách hàng không bao giờ quên. Nó có thể trở thành câu chuyện mà những thực khách tương lai tìm đọc và lưu tâm.

Những nhà hàng với lượng lớn đánh giá hữu ích thường có khả năng tạo ra sự gắn kết cộng đồng. Họ trở thành địa điểm mà khách hàng cảm thấy thoải mái, muốn quay lại và sẵn sàng đưa ra phản hồi để giúp cải thiện hoặc khen ngợi dịch vụ. Từ góc độ kinh doanh, tổng số đánh giá hữu ích cao còn là minh chứng cho việc nhà hàng đã tạo ra trải nghiệm đủ tốt và đáng nhớ đến mức mọi người muốn chia sẻ và giúp đỡ cộng đồng bằng những lời khuyên thực tế.

Với dữ liệu này, các nhà hàng khác có thể học hỏi từ những câu chuyện thành công này để cải thiện và tiếp cận gần hơn với cộng đồng. Những bài đánh giá hữu ích không chỉ là nguồn động viên mà còn là công cụ giúp doanh nghiệp phát triển, khẳng định vị thế trong lòng thực khách.